In [ ]:
import random
import numpy as np
import pandas as pd

import shapely

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')

import itertools
import copy

import time

<ipython-input-2-4d742dfb2c19>:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
CARD_SIZE = 100
NUMBER_UAVS = 100
NUMBER_TARGETS = 100
MAX_MODEL = 6

RANDOM_SEED = 1
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [ ]:
def log(x):
    return np.log(x) if x > 0 else -np.inf

In [ ]:
class Obj:
    min_x, max_x = x = (0, CARD_SIZE)
    min_y, max_y = y = (10, 20)
    min_z, max_z = z = (0, CARD_SIZE)

    def __init__(self, cord = None):
        if cord is None:
            self.set_random_cord()
        else:
            self.set_cord(cord)
        self.model = np.random.randint(0, MAX_MODEL)

    def set_random_cord(self):
        self.set_cord((np.random.randint(self.min_x, self.max_x), np.random.randint(self.min_y, self.max_y), np.random.randint(self.min_z, self.max_z)))

    def set_cord(self, cord):
        self.cord = cord

    def __str__(self):
        return f'''Координаты : {self.cord},\nМодель: {self.model}'''


In [ ]:
class Target(Obj):
    id_iter = itertools.count()

    def __init__(self, cord = None):
        super().__init__(cord)
        self.numbear = next(self.id_iter)

    def __str__(self):
        return f'''Цель {self.numbear}\nКоординаты : {self.cord},\nМодель: {self.model}'''

In [ ]:
class UAV(Obj):
    id_iter = itertools.count()

    def __init__(self, cord = None, velocity = None):
        super().__init__(cord)
        self.numbear = next(self.id_iter)

        if velocity is None:
            self.v = np.random.randint(1, round(CARD_SIZE/3))
        else:
            self.v = velocity


    def __str__(self):
        return f'''UAV {self.numbear}\nКоординаты : {self.cord},\nМодель: {self.model}'''

In [ ]:
EFFECTIVENESS_INTERACTION = np.zeros((MAX_MODEL, MAX_MODEL))
for i in range(MAX_MODEL):
    for j  in range(MAX_MODEL):
        if i <= j:
            EFFECTIVENESS_INTERACTION[i, j] = 1 / abs( -i + j  + 2) ** 2
        elif i > j:
            EFFECTIVENESS_INTERACTION[i, j] = i - j + MAX_MODEL - 1

EFFECTIVENESS_INTERACTION

array([[ 0.25      ,  0.11111111,  0.0625    ,  0.04      ,  0.02777778,
         0.02040816],
       [ 6.        ,  0.25      ,  0.11111111,  0.0625    ,  0.04      ,
         0.02777778],
       [ 7.        ,  6.        ,  0.25      ,  0.11111111,  0.0625    ,
         0.04      ],
       [ 8.        ,  7.        ,  6.        ,  0.25      ,  0.11111111,
         0.0625    ],
       [ 9.        ,  8.        ,  7.        ,  6.        ,  0.25      ,
         0.11111111],
       [10.        ,  9.        ,  8.        ,  7.        ,  6.        ,
         0.25      ]])

In [ ]:
class MapArea:
    EFFECTIVENESS_INTERACTION = EFFECTIVENESS_INTERACTION
    def __init__(self, uavs = None, targets = None):
        if uavs is None:
            self.set_random_uavs()
        else:
            self.uavs = uavs

        if targets is None:
            self.set_random_targets()
        else:
            self.targets = targets

    def set_random_uavs(self):
        self.uavs = [UAV() for _ in range(NUMBER_UAVS)]

    def set_random_targets(self):
        self.targets = [Target() for _ in range(NUMBER_TARGETS)]

In [ ]:
def g1(solution, parameters):
    """
    Время достижения цели для БЛА

    """
    def distance(point1, point2):
        return np.sqrt(sum([(point2[i] - point1[i]) ** 2 for i in range(3)]))

    str_out = getattr(solution, 'str')
    index = parameters['index']
    t = np.zeros(NUMBER_UAVS)

    for i_uav, i_target in enumerate(solution.distribution):
        if i_uav in index:
            if i_target >= 0:
                t[i_uav] = distance(solution.MAP.uavs[i_uav].cord, solution.MAP.targets[i_target].cord) / solution.MAP.uavs[i_uav].v
    str_out += f't: {t}\n'
    setattr(solution, 'str', str_out)
    return max(t)

def g2(solution, parameters):
    """
    Произведение значений эффективности БЛА назначенных на определенные цели

    """
    str_out = getattr(solution, 'str')
    index = parameters['index']

    res = np.ones(NUMBER_TARGETS)
    for i_uav, i_target in enumerate(solution.distribution):
        if i_uav in index:
            if i_target >= 0:
                if res[i_target] == 1:
                    res[i_target] =  solution.MAP.EFFECTIVENESS_INTERACTION[solution.MAP.uavs[i_uav].model][solution.MAP.targets[i_target].model]

    str_out += f'score: {res}\n'
    setattr(solution, 'str', str_out)
    return np.prod(res)

def g3(solution, parameters):
    """
    Количество задействованных при этом БЛА

    """
    str_out = getattr(solution, 'str')
    index = parameters['index']
    str_out += f'Количесво задействоанных UAV: {sum([1 for i in index if solution.distribution[i] > -1])}\n'
    setattr(solution, 'str', str_out)
    return sum([1 for i in index if solution.distribution[i] > -1])

In [ ]:
def f1(solution, parameters):
    """
    intersection_trajectories

    """
    def are_segments_intersecting(segment1, segment2):
        def orientation(p, q, r):
            val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
            if val == 0:
                return 0
            return 1 if val > 0 else 2

        def on_segment(p, q, r):
            if (q[0] <= max(p[0], r[0]) and q[0] >= min(p[0], r[0]) and
                    q[1] <= max(p[1], r[1]) and q[1] >= min(p[1], r[1])):
                return True
            return False

        p1, q1 = segment1
        p2, q2 = segment2

        o1 = orientation(p1, q1, p2)
        o2 = orientation(p1, q1, q2)
        o3 = orientation(p2, q2, p1)
        o4 = orientation(p2, q2, q1)

        if (o1 != o2 and o3 != o4) or \
                (o1 == 0 and on_segment(p1, p2, q1)) or \
                (o2 == 0 and on_segment(p1, q2, q1)) or \
                (o3 == 0 and on_segment(p2, p1, q2)) or \
                (o4 == 0 and on_segment(p2, q1, q2)):
            return True
        return False

    str_out = getattr(solution, 'str')
    index = parameters['index']
    segments = []
    for i_uav, i_target in enumerate(solution.distribution):
        if i_target >= 0:
            segments.append((solution.MAP.uavs[i_uav].cord, solution.MAP.targets[i_target].cord))

    flag = False
    for i in range(len(segments)):
        for j in range(i+1, len(segments)):
            flag = are_segments_intersecting(segments[i], segments[j])
            if flag:
                return flag

    setattr(solution, 'str', str_out)
    return not flag

def f2(solution, parameters):
    """


    """
    str_out = getattr(solution, 'str')
    index = parameters['index']

    f = {f'Цель {i} (Количестов нацеленных)':0 for i in range(NUMBER_TARGETS)}
    for i_uav, i_target in enumerate(solution.distribution):
        if i_uav in index:
            if i_target >= 0:
                f[f'Цель {i_target} (Количестов нацеленных)'] = f[f'Цель {i_target} (Количестов нацеленных)'] + 1

    str_out += f'Колисчесвто целей:{NUMBER_TARGETS}. Количесвто UAV:{NUMBER_UAVS}\nОграниечнеи выбор целей не > 1: {f}\n'
    setattr(solution, 'str', str_out)
    return sum([1 for i in range(NUMBER_TARGETS) if f[f'Цель {i} (Количестов нацеленных)'] > 1]) == 0

In [ ]:
CRITERIA = {
    'all/group':{
        'Время достижения цели для БЛА ' :[g1, 'min', {'index':[i for i in range(NUMBER_UAVS)]}],
        'произведение значений эффективности БЛА назначенных на определенные цели ' : [g2, 'max', {'index':[i for i in range(NUMBER_UAVS)]}],
        'Количество задействованных при этом БЛА  ' :[g3, 'min', {'index':[i for i in range(NUMBER_UAVS)]}],
    },
    'individual': {

    }
    # 'Название':[функция, 'min'/'max', [узлы на которые вещаем критерий]]
}

LIMITATIONS = {
    'all/group':{
        'Пересечение траекторий' :[f1, {'index':[i for i in range(NUMBER_UAVS)]}],
        'Выбор целей' :[f2, {'index':[i for i in range(NUMBER_UAVS)]}]
    },
    'individual': {
    }
    # 'Название':[функция, 'min'/'max', [узлы на которые вещаем критерий]]
}

In [ ]:
class Solution:
    MAP = MapArea()
    _CRITERIA = CRITERIA
    _LIMITATIONS = LIMITATIONS

    def __init__(self, distribution = None):
        if distribution is None:
            self.set_random_distribution()
        else:
            self.set_distribution(distribution)

        self.velocity = np.random.uniform(-1, 1, NUMBER_UAVS)
        self.best_position = copy.deepcopy(self)
        self.best_score = float('inf')

    def update_velocity(self, global_best_position, omega, phi_p, phi_g):
        rp, rg = np.random.random(), np.random.random()
        self.velocity = (omega * self.velocity +
                            phi_p * rp * (self.best_position.distribution - self.distribution) +
                            phi_g * rg * (global_best_position.distribution - self.distribution))

    def set_random_distribution(self, CONST_DISTRIBUTION={}):
        self.set_distribution(np.random.randint(-1, NUMBER_TARGETS, size=NUMBER_UAVS))

    def set_distribution(self, distr):
        self.distribution = distr
        setattr(self, 'str', "")
        self.calculation_limitations()
        self.calculation_objective_function()

    def update_distribution(self):
        disrt = np.clip(np.round(self.distribution + self.velocity), -1, NUMBER_TARGETS-1).astype(int)
        self.set_distribution(disrt)

    def calculation_objective_function(self):
        # Считаем все критрии вида all/group для объекта
        convolution = 1
        for k, v in self._CRITERIA['all/group'].items():

            value = v[0](self, v[2])
            value = value if not value == 0 else 1

            setattr(self, k, value)
            if v[1] == "max":
                convolution *= value
            else:
                convolution *= 1/value

        # Считаем все критрии для объекта
        for k, v in self._CRITERIA['individual'].items():
            for index_m in v['index_node']:

                value = v['f'][0](self, index_m, v['f'][2])
                value = value if not value == 0 else 1

                setattr(self, k + ' m' + str(index_m), value)
                if v['f'][1] == "max":
                    convolution *=     value
                else:
                    convolution *= 1 / value

        setattr(self, 'Свертка', convolution if getattr(self, 'Выполнение ограничеий') > 0 else -1)

    def calculation_limitations(self):
        # Считаем все критрии вида all/group для объекта
        convolution = 1
        for k, v in self._LIMITATIONS['all/group'].items():
            value = v[0](self, v[1])
            setattr(self, k, value)
            convolution *= value

        # Считаем все критрии для объекта
        for k, v in self._LIMITATIONS['individual'].items():
            for index_m in v['index_node']:
                value = v['f'][0](self, index_m, v['f'][1])
                setattr(self, k + ' m' + str(index_m), value)
                convolution *=  value

        setattr(self, 'Выполнение ограничеий', convolution)

    def __str__(self):
        return f'''Cвертка : {getattr(self, 'Свертка')},\nРаспределение: {self.distribution}\n{getattr(self, 'str')}'''

In [ ]:
Solution._CRITERIA = CRITERIA
Solution._LIMITATIONS = LIMITATIONS

In [ ]:
class PSO:
    def __init__(self, num_particles, omega, phi_p, phi_g) -> None:
        self.particles = self.generate_popul(num_particles)
        self.global_best_position = max(self.particles, key=lambda ind: getattr(ind,'Свертка'))
        self.global_best_score = getattr(self.global_best_position,'Свертка')
        self.omega = omega
        self.phi_p = phi_p
        self.phi_g = phi_g
        self.HISTORY = []

    def generate_popul(self, num_particles):
        res = [Solution(np.array([-1 for i in range(NUMBER_UAVS)]))]
        vars = [i for i in range(NUMBER_TARGETS)]
        for i_particle in range(num_particles-1):
            c = np.random.randint(1, NUMBER_TARGETS)
            arr = [-1 for i in range(NUMBER_TARGETS-c)] + list(np.random.choice(vars, c, replace=False))
            np.random.shuffle(arr)
            res.append(Solution(np.array(arr)))
        return res

    def optimize(self, max_f_calls):
        start_time = time.time()
        f_calls = len(self.particles)
        self.HISTORY.append([f_calls, time.time() - start_time, copy.deepcopy(self.particles), copy.deepcopy(self.global_best_position)])

        consecutive_iterations = 0
        previous_result =  log(self.global_best_score)
        self.iter = 0
        while f_calls <= max_f_calls:
            for particle in self.particles:
                score = getattr(particle,'Свертка')
                if score > particle.best_score:
                    particle.best_position = copy.deepcopy(particle)
                    particle.best_score = score
                if score > self.global_best_score:
                    self.global_best_position = copy.deepcopy(particle)
                    self.global_best_score = score
            for particle in self.particles:
                particle.update_velocity(self.global_best_position, self.omega, self.phi_p, self.phi_g)
                particle.update_distribution()
                f_calls += 1

            self.iter +=1

            self.HISTORY.append([f_calls, time.time() - start_time, copy.deepcopy(self.global_best_position)])
            current_result = log(self.global_best_score)

            if (previous_result is not None and abs(current_result - previous_result) < 1e-6) or current_result == previous_result:
                consecutive_iterations += 1
            else:
                consecutive_iterations = 0
            if consecutive_iterations == 50:
                pass
                # break  # Остановить цикл, если целевая функция не изменилась в течение 30 итераций подряд
            previous_result = log(self.global_best_score)
        return self.iter, self.global_best_score

    def print_res(self):
        print(self.global_best_score)

In [ ]:
# Определение диапазонов параметров
param_ranges = {
    'c1': (0.1, 2),  # диапазон значений для коэффициента ускорения c1
    'c2': (0.1, 2),  # диапазон значений для коэффициента ускорения c2
    'w': (0.1, 0.9),    # диапазон значений для инерционного веса w
    'num_particles': (10, 100),  # диапазон значений для количества частиц
}

for exp in range(10):
    RANDOM_SEED = exp
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)

    c1 = random.uniform(*param_ranges['c1'])
    c2 = random.uniform(*param_ranges['c2'])
    w = random.uniform(*param_ranges['w'])
    num_particles = random.randint(*param_ranges['num_particles'])
    experements = []
    f = 0
    for i in range(20):
        algo = PSO(num_particles, w, c1, c2)
        f, score = algo.optimize(10_000)
        print(f, score, "_".join(map(str, np.round([num_particles, c1, c2, w], 3))))
        print(algo.global_best_position)
        experements.append((f, algo.HISTORY))

    data = []
    for i in range(len(experements)):
        it, h = experements[i]
        for itr in h:
            res = getattr(itr[-1],'Свертка')
            temp = {
                'Коль-во частиц': num_particles,
                'MAX Количесво итераций': it,
                'c1_c2_w': '_'.join(map(str, [c1, c2, w])),
                '№ эксперемента':i+1,
                'Время':itr[1],
                'Вызовы ЦФ':itr[0],
                'Свертка':res if res > 0 else 0 ,
                'Распределение':itr[-1].distribution
            }
            data.append(temp)
    df = pd.DataFrame(data)
    df.to_csv(f'/content/drive/MyDrive/data/{NUMBER_UAVS}x{NUMBER_TARGETS}/1/{num_particles}_{"_".join(map(str, np.round([c1, c2, w], 3)))}_PSO.csv', index=False)

Выходные данные были обрезаны до нескольких последних строк (5000).
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 1.07043605 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 1.02056418 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
score: [1. 6. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 7. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1

In [ ]:
# Определение диапазонов параметров
param_ranges = {
    'c1': (0.1, 2),  # диапазон значений для коэффициента ускорения c1
    'c2': (0.1, 2),  # диапазон значений для коэффициента ускорения c2
    'w': (0.1, 0.9),    # диапазон значений для инерционного веса w
    'num_particles': (10, 100),  # диапазон значений для количества частиц
}

for exp in range(10):
    RANDOM_SEED = exp
    random.seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)

    c1 = 0.5 # random.uniform(*param_ranges['c1'])
    c2 = 1.5 # random.uniform(*param_ranges['c2'])
    w = 0.7 # random.uniform(*param_ranges['w'])
    num_particles = random.randint(*param_ranges['num_particles'])
    experements = []
    f = 0
    for i in range(20):
        algo = PSO(num_particles, w, c1, c2)
        f, score = algo.optimize(10_000)
        print(f, score, "_".join(map(str, np.round([num_particles, c1, c2, w], 3))))
        print(algo.global_best_position)
        experements.append((f, algo.HISTORY))

    data = []
    for i in range(len(experements)):
        it, h = experements[i]
        for itr in h:
            res = getattr(itr[-1],'Свертка')
            temp = {
                'Коль-во частиц': num_particles,
                'MAX Количесво итераций': it,
                'c1_c2_w': '_'.join(map(str, [c1, c2, w])),
                '№ эксперемента':i+1,
                'Время':itr[1],
                'Вызовы ЦФ':itr[0],
                'Свертка':res if res > 0 else 0 ,
                'Распределение':itr[-1].distribution
            }
            data.append(temp)
    df = pd.DataFrame(data)
    df.to_csv(f'/content/drive/MyDrive/data/{NUMBER_UAVS}x{NUMBER_TARGETS}/2/{num_particles}_{"_".join(map(str, np.round([c1, c2, w], 3)))}_PSO.csv', index=False)

Streaming output truncated to the last 5000 lines.
Cвертка : 1815.4413453452687,
Распределение: [-1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 22 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1  4 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 83
 -1 -1 -1 -1]
Колисчесвто целей:100. Количесвто UAV:100
Ограниечнеи выбор целей не > 1: {'Цель 0 (Количестов нацеленных)': 1, 'Цель 1 (Количестов нацеленных)': 1, 'Цель 2 (Количестов нацеленных)': 0, 'Цель 3 (Количестов нацеленных)': 0, 'Цель 4 (Количестов нацеленных)': 1, 'Цель 5 (Количестов нацеленных)': 0, 'Цель 6 (Количестов нацеленных)': 0, 'Цель 7 (Количестов нацеленных)': 0, 'Цель 8 (Количестов нацеленных)': 0, 'Цель 9 (Количестов нацеленных)': 0, 'Цель 10 (Количестов нацеленных)': 0, 'Цель 11 (Количестов нацеленных)': 0, 'Цель 12 (Количестов нацеленных)': 0, 'Цель 13 (Количестов нацеленных)': 0, '

In [ ]:
for uav in Solution.MAP.uavs:
    print(uav)

print()

for targets in Solution.MAP.targets:
    print(targets)

UAV 0
Координаты : (37, 18, 9),
Модель: 3
UAV 1
Координаты : (79, 10, 16),
Модель: 1
UAV 2
Координаты : (71, 16, 25),
Модель: 2
UAV 3
Координаты : (18, 14, 11),
Модель: 4
UAV 4
Координаты : (28, 12, 68),
Модель: 1
UAV 5
Координаты : (94, 10, 86),
Модель: 5
UAV 6
Координаты : (9, 17, 63),
Модель: 5
UAV 7
Координаты : (57, 11, 0),
Модель: 4
UAV 8
Координаты : (8, 18, 13),
Модель: 3
UAV 9
Координаты : (72, 17, 3),
Модель: 5
UAV 10
Координаты : (57, 13, 68),
Модель: 0
UAV 11
Координаты : (43, 14, 80),
Модель: 5
UAV 12
Координаты : (41, 12, 15),
Модель: 0
UAV 13
Координаты : (25, 12, 87),
Модель: 2
UAV 14
Координаты : (22, 19, 67),
Модель: 5
UAV 15
Координаты : (27, 15, 57),
Модель: 3
UAV 16
Координаты : (8, 10, 34),
Модель: 2
UAV 17
Координаты : (15, 17, 25),
Модель: 3
UAV 18
Координаты : (74, 10, 88),
Модель: 1
UAV 19
Координаты : (77, 13, 0),
Модель: 0
UAV 20
Координаты : (77, 16, 52),
Модель: 5
UAV 21
Координаты : (2, 15, 75),
Модель: 5
UAV 22
Координаты : (75, 14, 30),
Модель: 4
UAV 23

In [ ]:
print(Solution([-1 for i in range(NUMBER_UAVS)]))

Cвертка : 1.0,
Распределение: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
Колисчесвто целей:100. Количесвто UAV:100
Ограниечнеи выбор целей не > 1: {'Цель 0 (Количестов нацеленных)': 0, 'Цель 1 (Количестов нацеленных)': 0, 'Цель 2 (Количестов нацеленных)': 0, 'Цель 3 (Количестов нацеленных)': 0, 'Цель 4 (Количестов нацеленных)': 0, 'Цель 5 (Количестов нацеленных)': 0, 'Цель 6 (Количестов нацеленных)': 0, 'Цель 7 (Количестов нацеленных)': 0, 'Цель 8 (Количестов нацеленных)': 0, 'Цель 9 (Количестов нацеленных)': 0, 'Цель 10 (Количестов нацеленных)': 0, 'Цель 11 (Количестов нацеленных)': 0, 'Цель 12 (Количестов нацеленных)': 0, 'Цель 13 (